# Домашнее задание 2. По чуть-чуть про pandas, sklearn, kNN

В этом задании вы в миниатюре пройдете цикл решения задачи анализа данных от предобработки данных до построения модели и оценки качества. Вы будете решать задачу кредитного скоринга (вернет клиент кредит или нет) на примере небольшой выборки данных. 

В процессе решения задания вы поближе познакомитесь с библиотеками pandas, sklearn и тем, как работают классы в python, а также немного вспомните numpy, самостоятельно реализовав 1NN (это очень просто, потому что сам алгоритм очень простой).

Все задачи нужно выполнять исключительно самостоятельно! Если что-то непонятно, задавайте вопросы в чате. При обнаружении списывания ноль за задание ставится и тому, кто списал, и тому, у кого списали.

Также во всем задании запрещено использование циклов (исключение - второе бонусное задание)! Все операции необходимо выполнять средствами numpy, pandas и sklearn.

Разделение выборки на обучение и контроль производится случайно (если только вы не указали конкретный random_state), поэтому результаты при разных запусках ноутбука могут получаться разные.

Считывание данных уже сделано:

In [1]:
import numpy as np
import pandas as pd

In [2]:
tab = pd.read_csv("https://raw.githubusercontent.com/nadiinchi/iad2019/master/materials/scoring.csv")

Выведите первые несколько строчек датасета, посмотрите названия колонок:

In [3]:
tab.head()

,account,duration,credit_history,purpose,amount,savings,employment,installment_rate,guarantors,residence,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,furniture,3780,1,1,3,1,2,...,3,1,1,yes,0,0,1,0,male,1
1,1,24,2,car_new,1246,1,1,4,1,2,...,1,1,0,yes,1,0,0,0,male,0
2,0,15,2,car_used,3029,1,3,2,1,2,...,2,1,0,yes,0,0,1,0,male,1
3,0,36,3,television,4463,1,2,4,1,2,...,3,1,1,yes,0,0,1,0,male,0
4,-1,27,2,television,3416,1,2,3,1,2,...,3,1,0,yes,0,0,1,0,male,1


Сначала несколько вопросов просто так:
    
 __[0.5 балла]__ Какой процент клиентов моложе 30 лет (колонка age) снимает жилье (колонка housing_rent)? А какой процент клиентов старше 40 лет снимает жилье?

In [4]:
num = tab.shape[0]

crit1 = tab.age < 30
crit2 = tab.age > 40
rent = tab.housing_rent == 1

ans = tab.loc[crit1 & rent].shape[0]
print(ans/num*100, '% клиентов моложе 30 лет снимают жилье', sep='')

ans = tab.loc[crit2 & rent].shape[0]
print(ans/num*100, '% клиентов страше 40 лет снимают жилье', sep='')

12.166666666666668% клиентов моложе 30 лет снимают жилье
2.5% клиентов страше 40 лет снимают жилье


 __[0.5 балла]__ У скольких иностранцев (foreign == 'yes') число кредитов (credits_number) больше 2?

In [5]:
ans = tab.loc[(tab.foreign == 'yes') & (tab.credits_number > 2)].shape[0]
print('Ответ:', ans)

Ответ: 19


 __[0.5 балла]__ Выведите колонки job, life_insurance и guarantors для клиентов с номерами 105, 158, 306 и 336 одной таблицей (воспользуйтесь индексацией):

In [6]:
tab[['job', 'life_insurance', 'guarantors']].loc[[105, 158, 306, 336]]

,job,life_insurance,guarantors
105,2,1,1
158,2,0,1
306,3,0,0
336,2,1,1


 __[0.5 балла]__ Выведите среднюю сумму кредита (колонка amount) в каждой категории (колонка purpose), воспользовавшись методом groupby:

In [7]:
tab.groupby(tab.purpose).aggregate(np.mean).amount

purpose
business               4405.098361
car_new                3156.796178
car_used               5690.403509
domestic_appliances    1307.875000
education              3276.258065
furniture              3312.677966
others                 8681.700000
repairs                2792.769231
retraining             1156.000000
television             2554.237762
Name: amount, dtype: float64

Теперь по существу: нужно подготовить данные к построению модели.

 __[0.5 балла]__ Как вы видите, не все данные являются числовыми. С признаком purpose мы работать не будем - удалите его из датафрейма:

In [8]:
df = tab.copy()
df = df[df.columns.drop('purpose')]

In [9]:
df.shape[1] # сттолбец удалился

21

 __[0.5 балла]__ Необходимо перекодировать оставшиеся строковые значения в числа (закодировать пол female->0 и male->1 и колонку foreign числами yes->1 и no->0). Сделайте это с помощью pandas (без циклов, конечно же):

In [10]:
df.loc[(df.sex == 'female'),'sex'] = 0
df.loc[(df.sex == 'male'),'sex'] = 1

df.loc[(df.foreign == 'yes'),'foreign'] = 1
df.loc[(df.foreign == 'no'),'foreign'] = 0

 __[0.5 балла]__ Целевая переменная хранится в колонке target (последняя колонка). Являются ли классы сбалансированными (одинаковое число 0 и 1)? Воспользуйтесь методом [value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).
 
 **Ответ:** классы сбалансированы, число 0 и 1 одинаковое (300 и 300)

In [11]:
df.target.value_counts()

1    300
0    300
Name: target, dtype: int64

Выведем статистику датасета:

In [12]:
df.describe()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,credits_number,job,maintenance_people,telephone,real_estate,life_insurance,car,housing_rent,target
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,0.083333,21.878333,2.410000,3470.491667,1.200000,2.348333,2.988333,0.901667,2.831667,35.181667,1.366667,1.906667,1.150000,0.391667,0.268333,0.205000,0.343333,0.186667,0.500000
std,0.915306,12.749161,1.098747,3090.052187,0.924531,1.208069,1.131517,0.298013,1.103945,11.416182,0.565174,0.662334,0.357369,0.488530,0.443462,0.404038,0.475218,0.389969,0.500417
min,-1.000000,4.000000,0.000000,250.000000,0.000000,0.000000,1.000000,0.000000,1.000000,19.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,12.000000,2.000000,1363.250000,1.000000,2.000000,2.000000,1.000000,2.000000,26.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,18.000000,2.000000,2347.000000,1.000000,2.000000,3.000000,1.000000,3.000000,33.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
75%,1.000000,27.000000,4.000000,4318.750000,1.000000,3.000000,4.000000,1.000000,4.000000,41.000000,2.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,2.000000,72.000000,4.000000,18424.000000,4.000000,4.000000,4.000000,1.000000,4.000000,75.000000,4.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


 __[0.5 балла]__ Проанализируйте таблицу. Есть ли признаки с пропущенными значениями? (ответ необходимо писать в этой же ячейке; к ответу сделайте пояснение)
 
 **Ответ:** признаков с пропущенными значениями нет, так как количество значений всех признаков одинаково и равно 600 (для этого был использован вывод уникальных значений в ячейке ниже)

In [13]:
desc = df.describe()
desc.loc['count'].unique()

array([600.])

 __[0.5 балла]__ Запишите в переменную X матрицу объекты-признаки (выделите все колонки признаков, кроме target), а в переменную y - вектор правильных ответов (колонка target). К обоим объектам припишите .values, чтобы "достать" numpy-матрицу из датафрейма: далее мы везде будем работать с numpy-матрицами.

In [14]:
X = df[df.columns.drop('target')].values
y = df.target.values

 __[0.5 балла]__ С помощью sklearn-функции train_test_split разделите данные на обучение и контроль (пропорцию оставьте по умолчанию). Ячейка ?train_test_split покажет документацию функции (с примерами использования), также можно загуглить документацию.

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
#?train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

Выведите форму (shape) обучающей и контрольной выборок:

In [18]:
print(X_train.shape, X_test.shape)

(450, 20) (150, 20)


 __[0.5 балла]__ Как вы видели выше, все колонки имеют разный масштаб, это будет мешать нам при запуске метрического классификатора. Отнормируйте данные с помощью StandardScaler (интерфейс такой же, как мы изучали на семинаре):

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
#?StandardScaler

In [21]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

 __[0.5 балла]__ Теперь обучите классификатор 1-NN из sklearn на обучающей выборке, сделайте предсказания отдельно для обучающей и для контрольной выборки, на каждой выборке посчитайте точность (accuracy) с помощью accuracy_score. При создании объекта KNeighborsClassifier необходимо указать число соседей 1 (по умолчанию оно больше). 

Интерфейс KNeighborsClassifier такой же, как мы изучали на семинаре, а accuracy-score принимает на вход вектор предсказаний и вектор правильных ответов.

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [23]:
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(X_train_std, y_train) # обучаю классификатор

KNeighborsClassifier(n_neighbors=1)

In [24]:
pred_train = model.predict(X_train_std) # предсказываю для обучающей

In [25]:
pred_test = model.predict(X_test_std) # предсказываю для контрольной

In [26]:
accuracy_score(y_train, pred_train)

1.0

In [27]:
accuracy_score(y_test, pred_test)

0.6133333333333333

 __[4 балла,  включая my_accuracy и тестирование реализации]__ Наконец, реализуйте свой 1NN, воспользовавшись шаблоном ниже. Ваш класс должен уметь работать в двух режимах: в евклидовой метрикой и с манхэттанской метрикой (формулы метрик ниже). Напоминаем принцип работы 1NN: чтобы сделать предсказание для объекта, он выбирает ближайший объект из обучения и возвращает такой же класс, как у него.
* в методе \__init\__ необходимо запомнить, с какой метрикой мы работаем
* в методе fit нужно запомнить выборку - больше при обучении kNN ничего делать не нужно
* в методе predict нужно сделать три шага:
    * сначала построить матрицу расстояний между контрольными и обучающими объектами. Для этого можно использовать функцию [cdist](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html) (прочитайте ее документацию).
    * затем взять argmin по строкам - получится вектор индексов ближайших обучающих объектов для каждого контрольного объекта
    * выбрать компоненты y_train, отвечающие компонентам вектора индексов. Иными словами, вернуть те же метки классов, что у ближайших обучающих объектов.
    
Формулы метрик (расстояний):
* евклидова: $$\rho(x, y) = \sqrt{\sum_{i=1}^d (x_i - y_i)^2}$$
* манхэттанская: $$\rho(x, y) = \sum_{i=1}^d |x_i - y_i|$$

Модуль в numpy вычисляется как np.abs.

In [28]:
from scipy.spatial.distance import cdist

In [29]:
class MyKNN:
    def __init__(self, metric="euclid"): #По умолчанию евклидова, любой друой ввод - 
                                         # - манхэттенская (чтобы не выдавал ошибку)
        self.metric = metric         
        
    def fit(self, X_tr, y_tr):
        self.X_tr = X_tr
        self.y_tr = y_tr
        
    def predict(self, X_te):
        
        if self.metric == 'euclid':
            self.dist = cdist(X_te, self.X_tr, 'euclidean')
        else:
            self.dist = cdist(X_te, self.X_tr, 'cityblock')
            
        nearest_idx = np.argmin(self.dist, axis=1)
        ans = self.y_tr[nearest_idx]
        return ans
           

Реализуйте также функцию вычисления точности - доли правильных ответов:

In [31]:
def my_accuracy(y_pred, y_test):
    return np.mean(y_pred == y_test)

Пришло время запустить ваш классификатор и вычислить точность с помощью вашей функции. Обучите классификатор, сделайте предсказания для обучающей и контрольной выборок и посчитайте точность в обоих случаях. Сделайте это два раза - ддя евклидовой и манхэттанской метрик. Для евклидовой метрики цифры должны получиться точно такие же, как для реализации sklearn.

**Запуск классификатора номер 1 (евклидова метрика)**

In [32]:
my_model = MyKNN() # Евклидова по умолчанию
my_model.fit(X_train_std, y_train)    

my_pred_train = my_model.predict(X_train_std)
my_pred_test = my_model.predict(X_test_std)

In [33]:
my_accuracy(y_train, my_pred_train)

1.0

In [34]:
my_accuracy(y_test, my_pred_test)

0.6133333333333333

Доля правильных ответов совпадает с той, что получилась при реализации sklearn

**Запуск классификатора номер 2 (манхэттенская метрика)**

In [35]:
my_model = MyKNN(metric ='manhattan')
my_model.fit(X_train_std, y_train)    

my_pred_train = my_model.predict(X_train_std)
my_pred_test = my_model.predict(X_test_std)

In [36]:
my_accuracy(y_train, my_pred_train)

1.0

In [37]:
my_accuracy(y_test, my_pred_test)

0.6066666666666667

### Бонусное задание: реализация kNN
Добавьте в ваш KNN параметр k (число соседей) и реализуйте k-NN, который возвращает самый популярный класс среди k ближайших соседей объекта. Если число ближайших объектов для разных классов совпадает, возвращайте класс с меньшим номером. За реализацию, работающую для любого числа классов - 3 балла (посмотрите функции np.bincount, np.apply_along_axis (к сожалению, bincount не применяется построчно), np.argpartition), за реализацию, работающую только для двух классов - 2 балла. Не забудьте протестировать свой алгоритм :) В этом задании циклы использовать нельзя!


**Работает для двух классов**

In [38]:
class My_KNN:
    def __init__(self, metric='euclid', k=1): #По умолчанию евклидова метрика и 1 сосед
        self.metric = metric
        self.k = k
        
    def fit(self, X_tr, y_tr):
        self.X_tr = X_tr
        self.y_tr = y_tr
        
    def predict(self, X_te):
        
        if self.metric == 'euclid':
            self.dist = cdist(X_te, self.X_tr, 'euclidean')
        else:
            self.dist = cdist(X_te, self.X_tr, 'cityblock')
            
        indicies = np.argsort(self.dist)[:, :self.k]
        classes = self.y_tr[indicies]
        
        class_det = np.mean(classes, axis = 1)
        
        class_det[np.logical_or(class_det == 0.5, class_det < 0.5)] = 0 
        class_det[class_det > 0.5] = 1
        ans = class_det.astype(int)
        
        return ans
           

### Тесты

**Тест 1.** Параметры по умолчанию: евклидова метрика и k=1

In [39]:
my_model_k = My_KNN()
my_model_k.fit(X_train_std, y_train)    

my_pred_train_k = my_model_k.predict(X_train_std)
my_pred_test_k = my_model_k.predict(X_test_std)

In [40]:
my_accuracy(y_train, my_pred_train_k)

1.0

In [41]:
my_accuracy(y_test, my_pred_test_k)

0.6133333333333333

Accuracy такая же, как с использованием sklearn и MyKNN - вроде как работает

**Тест 2.** Манхэттенская метрика и три разных k

In [42]:
my_metric = 'manhattan'

# Первый k

my_k = 2

my_model_k = My_KNN(metric=my_metric, k=my_k)
my_model_k.fit(X_train_std, y_train)    

my_pred_train_k = my_model_k.predict(X_train_std)
my_pred_test_k = my_model_k.predict(X_test_std)

acc_train = my_accuracy(y_train, my_pred_train_k)
acc_test = my_accuracy(y_test, my_pred_test_k)

print('Доля верных ответов на обучающей выборке для k=', my_k, ': ', acc_train, sep='')
print('Доля верных ответов на тестовой выборке для k=', my_k, ': ', acc_test, sep='', end='\n\n')


# Второй k

my_k = 5

my_model_k = My_KNN(metric=my_metric, k=my_k)
my_model_k.fit(X_train_std, y_train)    

my_pred_train_k = my_model_k.predict(X_train_std)
my_pred_test_k = my_model_k.predict(X_test_std)

acc_train = my_accuracy(y_train, my_pred_train_k)
acc_test = my_accuracy(y_test, my_pred_test_k)

print('Доля верных ответов на обучающей выборке для k=', my_k, ': ', acc_train, sep='')
print('Доля верных ответов на тестовой выборке для k=', my_k, ': ', acc_test, sep='',end='\n\n')

# Третий k

my_k = 11

my_model_k = My_KNN(metric=my_metric, k=my_k)
my_model_k.fit(X_train_std, y_train)    

my_pred_train_k = my_model_k.predict(X_train_std)
my_pred_test_k = my_model_k.predict(X_test_std)

acc_train = my_accuracy(y_train, my_pred_train_k)
acc_test = my_accuracy(y_test, my_pred_test_k)

print('Доля верных ответов на обучающей выборке для k=', my_k, ': ', acc_train, sep='')
print('Доля верных ответов на тестовой выборке для k=', my_k, ': ', acc_test, sep='', end='\n\n')



Доля верных ответов на обучающей выборке для k=2: 0.8311111111111111
Доля верных ответов на тестовой выборке для k=2: 0.5666666666666667

Доля верных ответов на обучающей выборке для k=5: 0.8
Доля верных ответов на тестовой выборке для k=5: 0.62

Доля верных ответов на обучающей выборке для k=11: 0.7555555555555555
Доля верных ответов на тестовой выборке для k=11: 0.6333333333333333



**Тест 3.** Евклидова метрика и три разных k

In [43]:
my_metric = 'euclid'

# Первый k

my_k = 3

my_model_k = My_KNN(metric=my_metric, k=my_k)
my_model_k.fit(X_train_std, y_train)    

my_pred_train_k = my_model_k.predict(X_train_std)
my_pred_test_k = my_model_k.predict(X_test_std)

acc_train = my_accuracy(y_train, my_pred_train_k)
acc_test = my_accuracy(y_test, my_pred_test_k)

print('Доля верных ответов на обучающей выборке для k=', my_k, ': ', acc_train, sep='')
print('Доля верных ответов на тестовой выборке для k=', my_k, ': ', acc_test, sep='', end='\n\n')


# Второй k

my_k = 9

my_model_k = My_KNN(metric=my_metric, k=my_k)
my_model_k.fit(X_train_std, y_train)    

my_pred_train_k = my_model_k.predict(X_train_std)
my_pred_test_k = my_model_k.predict(X_test_std)

acc_train = my_accuracy(y_train, my_pred_train_k)
acc_test = my_accuracy(y_test, my_pred_test_k)

print('Доля верных ответов на обучающей выборке для k=', my_k, ': ', acc_train, sep='')
print('Доля верных ответов на тестовой выборке для k=', my_k, ': ', acc_test, sep='',end='\n\n')

# Третий k

my_k = 10

my_model_k = My_KNN(metric=my_metric, k=my_k)
my_model_k.fit(X_train_std, y_train)    

my_pred_train_k = my_model_k.predict(X_train_std)
my_pred_test_k = my_model_k.predict(X_test_std)

acc_train = my_accuracy(y_train, my_pred_train_k)
acc_test = my_accuracy(y_test, my_pred_test_k)

print('Доля верных ответов на обучающей выборке для k=', my_k, ': ', acc_train, sep='')
print('Доля верных ответов на тестовой выборке для k=', my_k, ': ', acc_test, sep='', end='\n\n')



Доля верных ответов на обучающей выборке для k=3: 0.8088888888888889
Доля верных ответов на тестовой выборке для k=3: 0.6133333333333333

Доля верных ответов на обучающей выборке для k=9: 0.7488888888888889
Доля верных ответов на тестовой выборке для k=9: 0.6133333333333333

Доля верных ответов на обучающей выборке для k=10: 0.7488888888888889
Доля верных ответов на тестовой выборке для k=10: 0.66

